<a href="https://colab.research.google.com/github/abhi-11nav/Malaria-Image-Detection/blob/main/Malaria_cell_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
# Importing the necessary libraries for building the model 

import tensorflow as tf 
import keras

from keras.layers import Dense, Flatten
from keras.models import Model 
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from keras.applications.resnet import ResNet50

from glob import glob


# Import visualization libraries 

import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns

In [ ]:
# Checking the GPU version and info

!nvidia-smi

Sat Aug  6 18:50:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Declaring the standard image size 

image_size = (224,224, 3)

In [ ]:
# Loading the resnet model 

model = ResNet50(input_shape=image_size,weights="imagenet",include_top=False)

In [ ]:
# We are not going to the train the pre trained layers in the ResNet50 Model. We are going to use the imagenet weights

for layers in model.layers[:10]:
  layers.trainable = False

In [ ]:
# Storing the number of outputs classes in a variable 

output_class_count = len(glob("/content/drive/MyDrive/cell_images/*"))

In [ ]:
print("Number of output classes:",output_class_count)

Number of output classes: 2


In [ ]:
# Completing the model with adding a fully connected and output layer at the end 

# First fully connected layer 
fc_layer1 = Flatten()(model.output)

# Prediction_layer / Output layer 
prediction_layer = Dense(2, activation="sigmoid")(fc_layer1)

In [ ]:
# Creating a model object and passing the inputs and outputs argument

model = Model(inputs = model.input, outputs=prediction_layer)

In [ ]:
# Printing the summary of the model 

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
# Installing split-folder to seperate test and train sets
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# importing splitfolders

import splitfolders

In [ ]:
# Splitting the data into train and validation sets

input_folder_path = "/content/drive/MyDrive/cell_images/Train"

output_folder_path = "/content/drive/MyDrive/cell_images/Test"

splitfolders.ratio(input_folder_path, output=output_folder_path, seed=42, ratio=(.75,.25))

Copying files: 27579 files [14:32, 31.59 files/s] 


In [34]:
# Calling the compile function on our final model 

model.compile(optimizer="adam",metrics=['accuracy'],loss="binary_crossentropy")

In [35]:
# Specifying the path of the dataset

train_path = "/content/drive/MyDrive/cell_images/Data/train"

test_path  = "/content/drive/MyDrive/cell_images/Data/val"

In [36]:
train_datagen = ImageDataGenerator(shear_range=0.2,zoom_range=0.2,horizontal_flip=True, vertical_flip = True, rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

In [37]:
training_data = train_datagen.flow_from_directory(train_path, target_size = (224,224), batch_size=16, class_mode="categorical")

testing_data = test_datagen.flow_from_directory(test_path, target_size=(224,224), class_mode="categorical", batch_size=16)

Found 20682 images belonging to 2 classes.
Found 6896 images belonging to 2 classes.


In [38]:
# Fitting the data to the model 

model.fit(training_data, validation_data = testing_data, epochs = 10)

Epoch 1/10
1293/1293 [==============================] - 444s 328ms/step - loss: 0.4770 - accuracy: 0.9034 - val_loss: 2192.3591 - val_accuracy: 0.5000
Epoch 2/10
1293/1293 [==============================] - 409s 316ms/step - loss: 0.2928 - accuracy: 0.9276 - val_loss: 205.6482 - val_accuracy: 0.5000
Epoch 3/10
1293/1293 [==============================] - 410s 317ms/step - loss: 0.2933 - accuracy: 0.9280 - val_loss: 1.7081 - val_accuracy: 0.5032
Epoch 4/10
1293/1293 [==============================] - 409s 316ms/step - loss: 0.1909 - accuracy: 0.9441 - val_loss: 1.8562 - val_accuracy: 0.5000
Epoch 5/10
1293/1293 [==============================] - 409s 316ms/step - loss: 0.1758 - accuracy: 0.9488 - val_loss: 15.4133 - val_accuracy: 0.5000
Epoch 6/10
1293/1293 [==============================] - 411s 317ms/step - loss: 0.3750 - accuracy: 0.9291 - val_loss: 0.8578 - val_accuracy: 0.5000
Epoch 7/10
1293/1293 [==============================] - 412s 318ms/step - loss: 0.3052 - accuracy: 0.9335 

In [39]:
plt.plot(model.history["loss"])
plt.plot(model.history["val_loss"])

TypeError: ignored

In [40]:
plt.plot(model.history["accuracy"])
plt.plot(model.history["val_accuracy"])

TypeError: ignored

In [ ]:
model.save("malaria_model.h5")